<a href="https://colab.research.google.com/github/Unknown-Geek/Blender-Cloud-Renderer/blob/main/Cloud_Blender_Renderer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# @title # Render Blender Files on Google Colab { display-mode: "form" }
# First, let's connect to Google Drive and install necessary dependencies
from google.colab import drive, files as colab_files
import os
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import glob
import re
import subprocess
import time
import sys

mount_drive = True

if mount_drive:
  drive.mount('/content/drive')
  print("Google Drive mounted successfully!")

# @title ## Blender Version Selection { display-mode: "form" }
# @markdown Choose which Blender version to use
blender_version = "4.1.0"  # @param ["2.93.5", "3.0.0", "3.1.0", "3.2.0", "3.3.0","3.6.0","4.0.0","4.1.0"]
reuse_downloaded = True # @param {type:"boolean"}
# @markdown Check if you've already downloaded Blender to your Google Drive

blender_folder_name = f"blender-{blender_version}-linux-x64"
blender_archive = f"{blender_folder_name}.tar.xz"
blender_urls = {
    "2.93.5": "https://download.blender.org/release/Blender2.93/blender-2.93.5-linux-x64.tar.xz",
    "3.0.0": "https://download.blender.org/release/Blender3.0/blender-3.0.0-linux-x64.tar.xz",
    "3.1.0": "https://download.blender.org/release/Blender3.1/blender-3.1.0-linux-x64.tar.xz",
    "3.2.0": "https://download.blender.org/release/Blender3.2/blender-3.2.0-linux-x64.tar.xz",
    "3.3.0": "https://download.blender.org/release/Blender3.3/blender-3.3.0-linux-x64.tar.xz",
    "3.6.0": "https://download.blender.org/release/Blender3.6/blender-3.6.0-linux-x64.tar.xz",
    "4.0.0": "https://download.blender.org/release/Blender4.0/blender-4.0.0-linux-x64.tar.xz",
    "4.1.0": "https://download.blender.org/release/Blender4.1/blender-4.1.0-linux-x64.tar.xz"
}

# Function to check if Blender extraction was successful
def check_blender_installation(folder_path):
  blender_executable = os.path.join(folder_path, "blender")
  return os.path.exists(blender_executable) and os.access(blender_executable, os.X_OK)

# Download or reuse Blender archive
if reuse_downloaded and os.path.exists(f"/content/drive/MyDrive/Blender/{blender_archive}"):
  # Copy previously downloaded Blender from Google Drive
  !mkdir -p /content/Blender
  !cp "/content/drive/MyDrive/Blender/{blender_archive}" "/content/{blender_archive}"
  print(f"Using previously downloaded Blender {blender_version}")
else:
  # Download Blender from repository
  print(f"Downloading Blender {blender_version}...")
  try:
    !wget {blender_urls[blender_version]}

    # Create Blender directory in Google Drive if needed
    !mkdir -p /content/drive/MyDrive/Blender

    # Save the downloaded archive for future use
    !cp "/content/{blender_archive}" "/content/drive/MyDrive/Blender/{blender_archive}"
    print(f"Blender {blender_version} has been saved to Google Drive for future use.")
  except Exception as e:
    print(f"Error downloading Blender: {e}")
    print("Please check your internet connection or try a different version.")
    sys.exit(1)

# Extract Blender
if not check_blender_installation(f"/content/{blender_folder_name}"):
  print(f"Extracting Blender {blender_version}...")
  try:
    !tar xf "{blender_archive}"
    # Verify extraction was successful
    if not check_blender_installation(f"/content/{blender_folder_name}"):
      print("Extraction seems incomplete. Trying again...")
      !rm -rf "/content/{blender_folder_name}"
      !tar xf "{blender_archive}"

      if not check_blender_installation(f"/content/{blender_folder_name}"):
        print("Failed to extract Blender properly. Please restart the runtime.")
        sys.exit(1)
    print("Extraction completed successfully!")
  except Exception as e:
    print(f"Error extracting Blender: {e}")
    sys.exit(1)
else:
  print(f"Blender {blender_version} is already extracted.")

# Fix libtcmalloc-minimal4 dependency
print("Setting up required libraries...")
os.environ["LD_PRELOAD"] = ""
try:
  !apt update
  !apt remove -y libtcmalloc-minimal4
  !apt install -y libtcmalloc-minimal4
except Exception as e:
  print(f"Warning: Issue with libtcmalloc installation: {e}")
  print("Continuing anyway, but rendering might be affected.")

# Check for the correct file path before setting LD_PRELOAD
lib_path = "/usr/lib/x86_64-linux-gnu/libtcmalloc_minimal.so.4"
if os.path.exists(lib_path):
  os.environ["LD_PRELOAD"] = lib_path
  print(f"Using library: {lib_path}")
else:
  # Try to find the correct file
  possible_libs = !find /usr/lib -name "libtcmalloc_minimal.so*"
  if possible_libs:
    os.environ["LD_PRELOAD"] = possible_libs[0]
    print(f"Using library: {possible_libs[0]}")
  else:
    print("Warning: Could not find libtcmalloc_minimal.so library")
    print("Rendering might be slower, but will still work.")

print("Libraries setup completed!")

# Check GPU status
print("\nChecking available GPU...")
try:
  gpu_info = !nvidia-smi
  if "T4" in ''.join(gpu_info):
    print("NVIDIA T4 GPU detected!")
  elif "not found" in ''.join(gpu_info):
    print("No GPU found. Rendering will use CPU only.")
  else:
    print("GPU detected, but not a T4. Performance may vary.")
except:
  print("Could not check GPU. Rendering will likely use CPU only.")
print("\nGPU check completed!")

clear_output()

# @title ## Blend File Selection { display-mode: "form" }

def find_blend_files(path='/content/drive'):
  """Find all .blend files in the given path recursively"""
  blend_files = []
  print("Searching for .blend files... (this may take a while)")
  try:
    # Use find command for faster searching
    result = !find {path} -name "*.blend" -type f
    if result:
      blend_files = [file for file in result if os.path.exists(file)]
  except Exception as e:
    print(f"Error using fast search: {e}")
    # Fallback to Python method
    for root, dirs, files in os.walk(path):
      for file in files:
        if file.endswith('.blend'):
          full_path = os.path.join(root, file)
          if os.path.exists(full_path):
            blend_files.append(full_path)
  return blend_files

def scan_drive_for_blend_files():
  """Scan Google Drive for .blend files and display them in a dropdown"""
  with output:
    clear_output()
    blend_files = find_blend_files()
    if blend_files:
      blend_file_dropdown.options = blend_files
      blend_file_dropdown.value = blend_files[0]
      print(f"Found {len(blend_files)} .blend files.")
    else:
      print("No .blend files found in Google Drive.")

def on_blend_file_change(change):
  """Handler for when the blend file selection changes"""
  if change['type'] == 'change' and change['name'] == 'value':
    with output:
      print(f"Selected: {change['new']}")

# Create widgets for blend file selection
output = widgets.Output()
scan_button = widgets.Button(description="Scan Drive")
blend_file_dropdown = widgets.Dropdown(
    options=[],
    description='Blend File:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='80%')
)

# Set up event handlers
scan_button.on_click(lambda b: scan_drive_for_blend_files())
blend_file_dropdown.observe(on_blend_file_change)

# Display widgets
print("Select a .blend file from your Google Drive:")
display(widgets.HBox([scan_button]))
display(blend_file_dropdown)
display(output)

# @title ## Render Settings { display-mode: "form" }
# @markdown Configure your render settings

# Render engine selection
render_engine = "CYCLES"  # @param ["CYCLES", "EEVEE"]

# Denoising settings
use_denoising = True  # @param {type:"boolean"}
denoiser_type = "OPENIMAGEDENOISE"

# Frame range settings - moved from separate section to here
render_type = "Animation"

# Function to update frame input visibility based on render type
def update_frame_inputs_visibility():
    if render_type == "Single Frame":
        frame_number_input.layout.display = None
        start_frame_input.layout.display = 'none'
        end_frame_input.layout.display = 'none'
    else:  # Animation
        frame_number_input.layout.display = 'none'
        start_frame_input.layout.display = None
        end_frame_input.layout.display = None

# Create frame input widgets
frame_number_input = widgets.IntText(
    value=10,
    description='Frame Number:',
    disabled=False
)

start_frame_input = widgets.IntText(
    value=1,
    description='Start Frame:',
    disabled=False
)

end_frame_input = widgets.IntText(
    value=10,
    description='End Frame:',
    disabled=False
)

# Create render type dropdown with handler
render_type_dropdown = widgets.Dropdown(
    options=["Single Frame", "Animation"],
    value=render_type,
    description='Render Type:',
    disabled=False
)

def on_render_type_change(change):
    """Handler for when render type changes"""
    if change['type'] == 'change' and change['name'] == 'value':
        global render_type
        render_type = change['new']
        update_frame_inputs_visibility()

render_type_dropdown.observe(on_render_type_change)

# Display render type and frame inputs
display(render_type_dropdown)
update_frame_inputs_visibility()  # Set initial visibility
display(frame_number_input)
display(start_frame_input)
display(end_frame_input)

# Resolution settings
resolution_x = 1080  # @param {type:"integer"}
resolution_y = 1080  # @param {type:"integer"}

# Quality settings
samples = 128  # @param {type:"integer"}
output_format = "PNG"  # @param ["PNG", "JPEG", "EXR", "TIFF", "WEBP"]

# T4 GPU specific settings
tile_size = 256  # @param {type:"integer"}
use_gpu = True  # @param {type:"boolean"}

# Output path
output_path = "/content/drive/MyDrive/Blender/output"  # @param {type:"string"}

# @title ## Start Rendering { display-mode: "form" }

def create_render_script(samples, resolution_x, resolution_y, tile_size, use_gpu):
  """Create a Python script to set rendering parameters"""
  script_content = f"""
import bpy
import os

# Set render engine
bpy.context.scene.render.engine = '{render_engine}'

# Set resolution directly
bpy.context.scene.render.resolution_x = {resolution_x}
bpy.context.scene.render.resolution_y = {resolution_y}
bpy.context.scene.render.resolution_percentage = 100  # Keep at 100% as we're setting exact resolution

# Configure rendering settings based on engine
if bpy.context.scene.render.engine == 'CYCLES':
    # Check if GPU is available
    gpu_available = False

    if {use_gpu}:
        try:
            # Get preferences
            cycles_prefs = bpy.context.preferences.addons['cycles'].preferences

            # Try different compute device types based on Blender version
            compute_device_types = ['CUDA', 'OPTIX']

            for device_type in compute_device_types:
                try:
                    cycles_prefs.compute_device_type = device_type
                    cycles_prefs.refresh_devices()

                    # Check for enabled devices
                    for dev in cycles_prefs.devices:
                        if dev.type == device_type and dev.use:
                            gpu_available = True
                            break

                    if gpu_available:
                        print(f"Using {{device_type}} for GPU rendering")
                        break
                except Exception as e:
                    print(f"Error trying {{device_type}}: {{e}}")
                    continue

            # Enable devices
            found_gpu = False
            for dev in cycles_prefs.devices:
                dev.use = False  # First disable all
                if dev.type in compute_device_types:
                    dev.use = True
                    found_gpu = True
                    print(f"Enabled GPU device: {{dev.name}}")

            if found_gpu:
                # Set cycles device to GPU
                bpy.context.scene.cycles.device = 'GPU'
                gpu_available = True
            else:
                print("No compatible GPU devices found, falling back to CPU")
                bpy.context.scene.cycles.device = 'CPU'
        except Exception as e:
            print(f"Error setting up GPU rendering: {{e}}")
            print("Falling back to CPU rendering")
            bpy.context.scene.cycles.device = 'CPU'
    else:
        print("GPU rendering disabled, using CPU")
        bpy.context.scene.cycles.device = 'CPU'

    # Set samples
    bpy.context.scene.cycles.samples = {samples}

    # Optimize for T4 GPU
    if gpu_available:
        try:
            # Older Blender versions use tile size
            if hasattr(bpy.context.scene.render, 'tile_x'):
                bpy.context.scene.render.tile_x = {tile_size}
                bpy.context.scene.render.tile_y = {tile_size}

            # Newer Blender versions may use these settings
            if hasattr(bpy.context.scene.cycles, 'tile_size'):
                bpy.context.scene.cycles.tile_size = {tile_size}

            # Some versions use this setting
            if hasattr(bpy.context.scene.cycles, 'use_auto_tile'):
                bpy.context.scene.cycles.use_auto_tile = False
        except Exception as e:
            print(f"Error setting tile size: {{e}}")

    # Configure denoising based on Blender version
    if {use_denoising}:
        print("Setting up denoising...")

        # Get Blender version info
        version_major = bpy.app.version[0]
        version_minor = bpy.app.version[1]
        print(f"Detected Blender version: {{version_major}}.{{version_minor}}")

        try:
            # Blender 4.x approach
            if version_major >= 4:
                # Main render denoising settings for Blender 4.x
                if hasattr(bpy.context.scene.cycles, 'use_denoising'):
                    bpy.context.scene.cycles.use_denoising = True
                    print("Enabled Cycles denoising")

                if hasattr(bpy.context.scene.cycles, 'denoiser'):
                    # Try to use the requested denoiser
                    try:
                        if '{denoiser_type}' == 'OPTIX' and gpu_available:
                            bpy.context.scene.cycles.denoiser = 'OPTIX'
                            print("Using OPTIX denoiser")
                        else:
                            bpy.context.scene.cycles.denoiser = 'OPENIMAGEDENOISE'
                            print("Using OpenImageDenoise")
                    except Exception as e:
                        print(f"Error setting denoiser type: {{e}}")
                        print("Using default denoiser")

                # Attempt to enable passes only if the attribute exists
                for view_layer in bpy.context.scene.view_layers:
                    try:
                        # This is the attribute that was causing the error
                        if hasattr(view_layer, 'use_pass_denoising_data'):
                            view_layer.use_pass_denoising_data = True
                            print("Enabled denoising data passes")
                    except Exception as e:
                        print(f"Could not enable denoising passes: {{e}}")

            # Blender 3.x approach
            elif version_major == 3:
                # Set up denoising for Blender 3.x
                if hasattr(bpy.context.scene.cycles, 'use_denoising'):
                    bpy.context.scene.cycles.use_denoising = True
                    print("Enabled Cycles denoising")

                if hasattr(bpy.context.scene.cycles, 'denoiser'):
                    # Try to use the requested denoiser
                    if '{denoiser_type}' == 'OPTIX' and gpu_available:
                        bpy.context.scene.cycles.denoiser = 'OPTIX'
                        print("Using OPTIX denoiser")
                    else:
                        bpy.context.scene.cycles.denoiser = 'OPENIMAGEDENOISE'
                        print("Using OpenImageDenoise")

                # Enable denoising data passes for compositing if available
                for view_layer in bpy.context.scene.view_layers:
                    # Safely check if attributes exist before using them
                    try:
                        if hasattr(view_layer, 'use_pass_denoising_data'):
                            view_layer.use_pass_denoising_data = True
                            print("Enabled denoising data passes")
                    except Exception as e:
                        print(f"Could not enable denoising passes: {{e}}")

            # Blender 2.8x-2.9x approach
            else:
                # View layer denoising for Blender 2.8x and 2.9x
                for view_layer in bpy.context.scene.view_layers:
                    if hasattr(view_layer.cycles, 'use_denoising'):
                        view_layer.cycles.use_denoising = True
                        print("Enabled view layer denoising")
                    if hasattr(view_layer.cycles, 'denoising_store_passes'):
                        view_layer.cycles.denoising_store_passes = True
                        print("Enabled denoising store passes")

                # For Blender 2.93, check for newer denoiser options
                if version_minor >= 93 and hasattr(bpy.context.scene.cycles, 'denoiser'):
                    if '{denoiser_type}' == 'OPTIX' and gpu_available:
                        bpy.context.scene.cycles.denoiser = 'OPTIX'
                        print("Using OPTIX denoiser")
                    else:
                        bpy.context.scene.cycles.denoiser = 'OPENIMAGEDENOISE'
                        print("Using OpenImageDenoise")
        except Exception as e:
            print(f"Error configuring denoising: {{e}}")
            print("Some denoising features may not be available with this Blender version")
    else:
        print("Denoising disabled")
        try:
            # Disable denoising for all Blender versions
            if hasattr(bpy.context.scene.cycles, 'use_denoising'):
                bpy.context.scene.cycles.use_denoising = False

            # For view layers in 2.8x-2.9x
            for view_layer in bpy.context.scene.view_layers:
                if hasattr(view_layer.cycles, 'use_denoising'):
                    view_layer.cycles.use_denoising = False
        except Exception as e:
            print(f"Error disabling denoising: {{e}}")

elif bpy.context.scene.render.engine == 'EEVEE':
    # EEVEE specific settings
    try:
        if hasattr(bpy.context.scene.eevee, 'taa_render_samples'):
            bpy.context.scene.eevee.taa_render_samples = {samples}
            print(f"Set EEVEE samples to {samples}")

        # Set up EEVEE denoising if available and requested
        if {use_denoising}:
            print("Setting up EEVEE quality improvements...")
            if hasattr(bpy.context.scene.eevee, 'use_gtao'):
                bpy.context.scene.eevee.use_gtao = True
            if hasattr(bpy.context.scene.eevee, 'use_ssr'):
                bpy.context.scene.eevee.use_ssr = True
            if hasattr(bpy.context.scene.eevee, 'use_ssr_refraction'):
                bpy.context.scene.eevee.use_ssr_refraction = True
            if hasattr(bpy.context.scene.eevee, 'use_taa_reprojection'):
                bpy.context.scene.eevee.use_taa_reprojection = True
            # Specific EEVEE denoise option in newer versions
            if hasattr(bpy.context.scene.eevee, 'use_denoise'):
                bpy.context.scene.eevee.use_denoise = True
                print("EEVEE denoising enabled")
    except Exception as e:
        print(f"Error configuring EEVEE settings: {{e}}")

try:
    # Set output format
    if '{output_format}' == 'PNG':
        bpy.context.scene.render.image_settings.file_format = 'PNG'
        bpy.context.scene.render.image_settings.color_depth = '8'
        bpy.context.scene.render.image_settings.compression = 15
    elif '{output_format}' == 'JPEG':
        bpy.context.scene.render.image_settings.file_format = 'JPEG'
        bpy.context.scene.render.image_settings.quality = 90
    elif '{output_format}' == 'EXR':
        bpy.context.scene.render.image_settings.file_format = 'OPEN_EXR'
        bpy.context.scene.render.image_settings.color_depth = '32'
    elif '{output_format}' == 'TIFF':
        bpy.context.scene.render.image_settings.file_format = 'TIFF'
        bpy.context.scene.render.image_settings.color_depth = '16'
    elif '{output_format}' == 'WEBP':
        bpy.context.scene.render.image_settings.file_format = 'WEBP'
        bpy.context.scene.render.image_settings.quality = 90

    print(f"Output format set to {output_format}")
except Exception as e:
    print(f"Error setting output format: {{e}}")
    print(f"Using default output format instead")

# Set output path
try:
    output_dir = "{output_path}"
    os.makedirs(output_dir, exist_ok=True)
    bpy.context.scene.render.filepath = output_dir + "/"
    print(f"Output directory set to: {{output_dir}}")
except Exception as e:
    print(f"Error setting output path: {{e}}")
    print("Using default output path instead")

print("Render settings applied successfully!")
"""

  with open('/content/render_settings.py', 'w') as f:
    f.write(script_content)

  return '/content/render_settings.py'

def start_rendering():
  if not blend_file_dropdown.value:
    with render_output:
      clear_output()
      print("Error: Please select a .blend file first!")
    return

  blend_file = blend_file_dropdown.value

  # Get frame values based on current render type
  if render_type == "Single Frame":
    frame_num = frame_number_input.value
    start_frame = frame_num  # For display in the output
    end_frame = frame_num    # For display in the output
  else:  # Animation
    start_frame = start_frame_input.value
    end_frame = end_frame_input.value
    frame_num = start_frame  # For single frame compatibility

  with render_output:
    clear_output()
    print(f"Starting render with the following settings:")
    print(f"Blend file: {blend_file}")
    print(f"Render engine: {render_engine}")
    print(f"Output format: {output_format}")
    print(f"Samples: {samples}")
    print(f"Resolution: {resolution_x}x{resolution_y}")

    # Display frame information based on render type
    if render_type == "Single Frame":
      print(f"Frame: {frame_num}")
    else:
      print(f"Frame range: {start_frame} - {end_frame}")

    if render_engine == "CYCLES":
      print(f"Denoising: {'Enabled' if use_denoising else 'Disabled'}")
      if use_denoising:
        print(f"Denoiser: {denoiser_type}")
      print(f"GPU rendering: {'Enabled' if use_gpu else 'Disabled'}")
      print(f"Tile size: {tile_size} (optimized for T4 GPU)")

    # Create output directory if it doesn't exist
    os.makedirs(output_path, exist_ok=True)

    # Create a Python script with render settings
    script_path = create_render_script(samples, resolution_x, resolution_y, tile_size, use_gpu)

    # Build the Blender command - don't override render engine in command line
    # as it's set in the Python script for better error handling
    cmd = f"cd /content && ./blender-{blender_version}-linux-x64/blender -b \"{blend_file}\" -noaudio -P {script_path}"

    # Add frame settings based on render type
    if render_type == "Single Frame":
      cmd += f" -f {frame_num}"
    else:  # Animation
      cmd += f" -s {start_frame} -e {end_frame} -a"

    print("\nExecuting command:")
    print(cmd)
    print("\nRender progress:")

    # Execute the command with better error handling
    try:
      start_time = time.time()
      process = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, universal_newlines=True)

      # Stream the output
      for line in process.stdout:
        print(line.strip())

      process.wait()

      end_time = time.time()
      render_time = end_time - start_time

      if process.returncode != 0:
        print(f"\nRendering failed with return code: {process.returncode}")
      else:
        print(f"\nRendering completed in {render_time:.2f} seconds!")
        print(f"Check your output at: {output_path}")
    except Exception as e:
      print(f"Error during rendering: {e}")

render_button = widgets.Button(
    description='Start Rendering',
    button_style='success',
    layout=widgets.Layout(width='200px', height='50px')
)

render_button.on_click(lambda b: start_rendering())

render_output = widgets.Output()

display(render_button)
display(render_output)

# @title ## View Rendered Output { display-mode: "form" }

def list_rendered_files():
  with view_output:
    clear_output()

    if not os.path.exists(output_path):
      print(f"Output directory {output_path} does not exist.")
      return

    # Get the appropriate file extension based on output format
    format_extensions = {
      "PNG": "png",
      "JPEG": "jpg",
      "EXR": "exr",
      "TIFF": "tiff",
      "WEBP": "webp"
    }

    extension = format_extensions.get(output_format, output_format.lower())
    files = sorted(glob.glob(f"{output_path}/*.{extension}"))

    if not files:
      print(f"No {output_format} files found in {output_path}")
      return

    print(f"Found {len(files)} rendered files:")

    # Show the most recent files (up to 5)
    for i, file in enumerate(files[-5:]):
      print(f"{i+1}. {os.path.basename(file)}")

    # Display the most recent rendered image if it's a supported web format
    latest_file = files[-1]
    basename = os.path.basename(latest_file)

    # Copy file to Colab session for viewing and downloading
    !cp "{latest_file}" "/content/{basename}"

    if output_format in ["PNG", "JPEG", "WEBP"]:
      display(HTML(f'<img src="/content/{basename}" style="max-width: 800px"/>'))
      print(f"\nShowing preview of: {basename}")
    else:
      print(f"\nCannot display preview for {output_format} files in the browser.")
      print(f"File has been copied to Colab session for downloading: {basename}")

    # Create download button
    download_button = widgets.Button(
      description=f'Download {basename}',
      button_style='info'
    )

    def on_download_click(b):
      colab_files.download(f"/content/{basename}")

    download_button.on_click(on_download_click)
    display(download_button)

view_button = widgets.Button(
    description='View Rendered Output',
    button_style='info',
    layout=widgets.Layout(width='200px')
)

view_button.on_click(lambda b: list_rendered_files())

view_output = widgets.Output()

display(view_button)
display(view_output)

Select a .blend file from your Google Drive:


Dropdown(description='Blend File:', layout=Layout(width='80%'), options=(), style=DescriptionStyle(description…

Output()

Dropdown(description='Render Type:', index=1, options=('Single Frame', 'Animation'), value='Animation')

IntText(value=10, description='Frame Number:', layout=Layout(display='none'))

...

IntText(value=10, description='End Frame:')

Button(button_style='success', description='Start Rendering', layout=Layout(height='50px', width='200px'), sty…

Output()

Button(button_style='info', description='View Rendered Output', layout=Layout(width='200px'), style=ButtonStyl…

Output()